In [ ]:
import itertools as it
import operator
import re
import subprocess
import tempfile
import time
from collections import Counter, defaultdict
from functools import partial
from pathlib import Path

import awkward as ak
import duckdb
import gfapy
import holoviews as hv
import ibis
import matplotlib.pyplot as plt
import numba
import numpy as np
import pandas as pd
import parasail
import pod5
import polars as pl
import pyabpoa
import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.dataset as ds
import pyfastx
import pysam
import spoa
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from pyarrow import csv
from pywfa import WavefrontAligner
from tqdm.auto import tqdm, trange

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.sequencing.align as align
import paulssonlab.sequencing.cigar as scigar
import paulssonlab.sequencing.consensus as con
import paulssonlab.sequencing.gfa as sgfa
import paulssonlab.sequencing.io as sio
import paulssonlab.sequencing.processing as processing
import paulssonlab.sequencing.uuid as uuid
from paulssonlab.util.sequence import reverse_complement

In [ ]:
hv.extension("bokeh")

In [ ]:
%load_ext pyinstrument
import line_profiler
import pyinstrument

%load_ext line_profiler

In [ ]:
pl.enable_string_cache()

# Config

In [ ]:
gfa_filename = "/home/jqs1/scratch3/jqs1/sequencing/230930_alignment_test/barcode.gfa"

In [ ]:
gfa = gfapy.Gfa.from_file(gfa_filename)

# Duplex pairing

## UUID parsing

In [ ]:
pod5_filename = "/home/jqs1/scratch3/jqs1/sequencing/230930_alignment_test/230707_repressilators/channel-135_merged.pod5"
gaf_filename = "/home/jqs1/scratch3/jqs1/sequencing/230930_alignment_test/230707_repressilators/channel-135_merged.gaf"

In [ ]:
%%time
# gaf = pa.Table.from_batches([next(sio.iter_gaf(gaf_filename))])
# gaf = pa.Table.from_batches(list(it.islice(sio.iter_gaf(gaf_filename), 2)))
gaf = pa.Table.from_batches(sio.iter_gaf(gaf_filename))

In [ ]:
%%time
gaf_simplex = gaf.filter(pc.invert(pc.match_substring(gaf["name"], ";")))

In [ ]:
name_col = gaf_simplex["name"]  # [:10]

In [ ]:
%%time
uuids = uuid.parse_uuids(uuid.remove_hyphens(ak.from_arrow(name_col)))

In [ ]:
x = np.concatenate([uuids] * 1000)
%timeit ak.to_arrow(ak.enforce_type(ak.from_numpy(x), "bytes"))

In [ ]:
x.nbytes / 1e6

In [ ]:
len(x)

In [ ]:
%timeit ak.to_arrow(ak.enforce_type(ak.from_numpy(uuids), "bytes"))

In [ ]:
ak.to_arrow(ak.enforce_type(ak.from_numpy(uuids), "bytes"))

In [ ]:
gaf_simplex["name"]

## Join

In [ ]:
uuids = uuid.parse_uuids(uuid.remove_hyphens(ak.from_arrow(gaf_simplex["name"])))
uuids_bytes = ak.to_arrow(
    ak.enforce_type(ak.from_numpy(uuids), "bytes"), extensionarray=False
)
# waiting on https://github.com/apache/arrow/issues/39232
# not sure if it's necessary/helps performance
# uuids_bytes = uuids_bytes.cast(pa.binary(16))
gaf_simplex_parsed = gaf_simplex.set_column(
    gaf_simplex.column_names.index("name"), "name", uuids_bytes
)

In [ ]:
reader = pod5.Reader(pod5_filename)

In [ ]:
run_info_table = reader.run_info_table.read_all()

In [ ]:
read_table = reader.read_table.read_all()

In [ ]:
read_table.column_names

In [ ]:
read_table2 = read_table.select(["read_id", "start", "well", "channel", "run_info"])

In [ ]:
read_table_df = pl.from_arrow(read_table2).lazy()

In [ ]:
gaf_df = pl.from_arrow(gaf_simplex_parsed).lazy()

In [ ]:
joined_df = read_table_df.join(gaf_df, left_on="read_id", right_on="name", how="left")

In [ ]:
joined_df = joined_df.collect()

In [ ]:
joined_df.columns

In [ ]:
joined_df.group_by("run_info")

In [ ]:
# first unique read_id
# same well, channel
# start within (1s * 5000 kHz)

In [ ]:
run_info = {}
for col in ["sample_rate"]:
    run_info[col] = dict(
        zip(
            run_info_table["acquisition_id"].to_pylist(),
            run_info_table[col].to_pylist(),
        )
    )

In [ ]:
run_info

In [ ]:
run_info_table.select(["acquisition_id", "sample_rate"]).to_pydict()

In [ ]:
sample_rate = 

In [ ]:
joined_df.rolling()

In [ ]:
read_table  # ["run_info"]